# Disparity Plot
David Shean  
January 2023

Used to visualize disparity components and magnitude for orthoimage offets (output of vmap)

In [ ]:
import os
import glob

import numpy as np
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar

import rasterio as rio
from osgeo import gdal

In [ ]:
from asp_plot_util import *

In [ ]:
disp_fn = '/Users/dshean/scr/BlackSky/Utqiagvik_20220425_stereo/WV_comparison/22APR17225148-P1BS-506304172010_01_P001_ortho__BSG-102-20220425-215147-22900061_georeferenced-pan_ortho_rpc_vmap_maxm_35px_spm1/22APR17225148-P1BS-506304172010_01_P001_ortho__BSG-102-20220425-215147-22900061_georeferenced-pan_ortho_rpc_vmap_maxm_35px_spm1-F.tif'
disp_fn = '/Users/dshean/scr/utqiagvik/WV02_20220411_10300100D102A400_10300100D1B7D600/ncc_dgmaprpc/ncc_dgmaprpc-RD.tif'

In [ ]:
#Specify output units
unit='pixels'
#unit='meters'
#Remove median offset in x and y
remove_bias = True
#Add quivers to offset magnitude plot to indicate direction
quiver = True

In [ ]:
#Extract ground sample distance
with rio.open(disp_fn) as ds:
    gsd = ds.transform[0]

In [ ]:
#Read band 1 (dx) and band 2 (dy) data
#TODO: load overview if present, prohibitively slow for large WV iamges
dx = read_array(disp_fn, 1)
dy = read_array(disp_fn, 2)

In [ ]:
#Scale offsets to meters
if unit == 'meters':
    dx *= gsd
    dy *= gsd

In [ ]:
#Remove median disparity
if remove_bias:
    dx_offset = np.ma.median(dx)
    dy_offset = np.ma.median(dy)
    print(dx_offset, dy_offset)
    dx -= dx_offset
    dy -= dy_offset

In [ ]:
#Compute magnitude
dm = np.sqrt(dx**2 + dy**2)

In [ ]:
#Compute robust colorbar limits (default is 2-98 percentile)
clim = get_clim(dm, perc=(2,98), symm=True)
clim

In [ ]:
f, axa = plt.subplots(1, 3, figsize=(10, 3), dpi=220)

plot_ar(dx, axa[0], cmap='RdBu', clim=clim, label=unit) #f'x offset ({unit})'
plot_ar(dy, axa[1], cmap='RdBu', clim=clim, label=unit)
plot_ar(dm, axa[2], cmap='inferno', clim=(0, clim[1]), label=unit)

#Add quiver vectors
if quiver:
    #Set ~30 points for quivers along x dimension
    stride = int(dx.shape[1]/30.)
    iy, ix = np.indices(dx.shape)[:, ::stride, ::stride]
    dx_q = dx[::stride, ::stride]
    dy_q = dy[::stride, ::stride]
    #plt.quiver(ix, -iy, dx_q*gsd, dy_q*gsd)
    axa[2].quiver(ix, iy, dx_q, dy_q, color='white')

#Add scalebar
sb = ScaleBar(gsd)
axa[0].set_title('x offset')
axa[1].set_title('y offset')
axa[2].set_title('offset magnitude')

plt.tight_layout()